In [2]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

topic_SP_list = ['THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
topic_list = ['Tân Hiệp Phát','Ban Lãnh Đạo Mới','THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
channel_list = ['Facebook', 'Fanpage', 'Forum', 'News']
channel_list_SP = ['Facebook', 'Fanpage', 'News', 'Youtube','Forum','Tiktok','Threads']
sent_list = ['Positive', 'Neutral', 'Negative']
labels_list = ['HOẠT ĐỘNG FANPAGE','BAN LÃNH ĐẠO','SẢN PHẨM','THƯƠNG HIỆU/CÔNG TY','CHƯƠNG TRÌNH TÀI TRỢ','QUẢNG CÁO','TUYỂN DỤNG','CHIẾN DỊCH SỰ KIỆN']

CORP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPnov.xlsx", sheet_name='CORP') #(7SP, THP, BLD)
SP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPnov.xlsx", sheet_name='SP') #(7SP - chỉnh negative)

In [3]:
 slide_n_dict = {}

In [4]:
#Slide 6

channel_topic = CORP.pivot_table(index='Topic', columns='Channel', values='Id',aggfunc = 'count').fillna(0)
channel_topic['Total'] = channel_topic.sum(axis=1)
channel_topic = channel_topic.sort_values(by='Total',ascending = False).reindex(topic_list).reset_index()
if 'Forum' not in channel_topic.columns:
    channel_topic['Forum'] = 0
channel_topic['Khác'] = channel_topic.drop(columns = channel_list+ ['Topic', 'Total']).sum(axis=1).fillna(0)
channel_topic = channel_topic[['Topic'] + channel_list + ['Khác','Total']] 
print(channel_topic)

slide_n_dict['Slide 6'] = channel_topic

Channel                           Topic  Facebook  Fanpage  Forum   News  \
0                         Tân Hiệp Phát     211.0   1585.0    2.0  103.0   
1                      Ban Lãnh Đạo Mới       NaN      NaN    NaN    NaN   
2                 THP Trà Xanh Không Độ     588.0     18.0    0.0   39.0   
3            THP Nước tăng Lực Number 1     889.0      6.0    0.0   16.0   
4                 THP Sữa Đậu Nành Soya     377.0      7.0    0.0   13.0   
5         THP Trà Thanh Nhiệt Dr. Thanh      72.0      0.0    1.0    0.0   
6        THP Trà Sữa Macchiato Không Độ       1.0      0.0    0.0    0.0   
7        THP Number 1 Active Chanh Muối      68.0      0.0    0.0  175.0   
8              THP Number 1 Chanh & Dâu      19.0      0.0    0.0    0.0   

Channel   Khác   Total  
0        309.0  2210.0  
1          0.0     NaN  
2        298.0   943.0  
3        154.0  1065.0  
4         54.0   451.0  
5         22.0    95.0  
6          2.0     3.0  
7          1.0   244.0  
8          1.0

In [5]:
#Slide 7

slide_n_dict['Slide 7'] = datemerge(CORP.groupby(['PublishedDate', 'Topic']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum', columns='Topic'), '2024-10-26', '2024-11-25')
print(slide_n_dict['Slide 7'])

               THP Number 1 Active Chanh Muối  THP Number 1 Chanh & Dâu  \
PublishedDate                                                             
2024-10-26                                4.0                       0.0   
2024-10-27                                7.0                       0.0   
2024-10-28                                8.0                       0.0   
2024-10-29                                6.0                       0.0   
2024-10-30                               17.0                       6.0   
2024-10-31                               15.0                       0.0   
2024-11-01                               16.0                       0.0   
2024-11-02                               10.0                       2.0   
2024-11-03                               41.0                       0.0   
2024-11-04                                5.0                       0.0   
2024-11-05                               10.0                       0.0   
2024-11-06               

In [6]:
#Silde 8

nsr_chart_df = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id', aggfunc='count').fillna(0)
nsr_chart_df['NSR'] = round((nsr_chart_df['Positive'] - nsr_chart_df['Negative']) / (nsr_chart_df['Positive'] + nsr_chart_df['Negative']), 3)
nsr_chart_df = nsr_chart_df.drop(sent_list, axis=1)

buzz_vertical_df = CORP.groupby('Topic').agg({'Id': 'count'}).pivot_table(index='Topic', values='Id', aggfunc='sum')

unique_voice_vertical_df = CORP.groupby('Topic').agg({'AuthorId': 'nunique'}).pivot_table(index='Topic', values='AuthorId', aggfunc='sum')

slide_8 = pd.concat([buzz_vertical_df, nsr_chart_df, unique_voice_vertical_df], axis=1).reindex(topic_list).fillna(0)
print(slide_8)
slide_n_dict['Slide 8'] = slide_8


                                    Id    NSR  AuthorId
Topic                                                  
Tân Hiệp Phát                   2210.0  0.875     660.0
Ban Lãnh Đạo Mới                   0.0  0.000       0.0
THP Trà Xanh Không Độ            943.0  1.000     445.0
THP Nước tăng Lực Number 1      1065.0  1.000     225.0
THP Sữa Đậu Nành Soya            451.0  1.000     243.0
THP Trà Thanh Nhiệt Dr. Thanh     95.0  1.000      40.0
THP Trà Sữa Macchiato Không Độ     3.0  0.000       3.0
THP Number 1 Active Chanh Muối   244.0  1.000      71.0
THP Number 1 Chanh & Dâu          20.0  0.000       6.0


In [7]:
#Silde 9

topic_sentiment = CORP.pivot_table(index='Topic',columns = 'Sentiment', values = 'Id',aggfunc = 'count')
print(topic_sentiment)
topic_sentiment['Total'] = topic_sentiment.sum(axis=1)
topic_sentiment = topic_sentiment.sort_values(by = 'Total', ascending = False).fillna(0).reset_index()
topic_sentiment = topic_sentiment[['Topic'] + sent_list + ['Total']]
print(topic_sentiment)

slide_n_dict['Slide 9'] = topic_sentiment

Sentiment                       Negative  Neutral  Positive
Topic                                                      
THP Number 1 Active Chanh Muối       NaN      9.0     235.0
THP Number 1 Chanh & Dâu             NaN     20.0       NaN
THP Nước tăng Lực Number 1           NaN   1042.0      23.0
THP Sữa Đậu Nành Soya                NaN    437.0      14.0
THP Trà Sữa Macchiato Không Độ       NaN      3.0       NaN
THP Trà Thanh Nhiệt Dr. Thanh        NaN     91.0       4.0
THP Trà Xanh Không Độ                NaN    896.0      47.0
Tân Hiệp Phát                       65.0   1169.0     976.0
Sentiment                           Topic  Positive  Neutral  Negative   Total
0                           Tân Hiệp Phát     976.0   1169.0      65.0  2210.0
1              THP Nước tăng Lực Number 1      23.0   1042.0       0.0  1065.0
2                   THP Trà Xanh Không Độ      47.0    896.0       0.0   943.0
3                   THP Sữa Đậu Nành Soya      14.0    437.0       0.0   451.0
4    

In [8]:
#Slide 10
df_list_CORP = []
for i in topic_list:
    brand = CORP[CORP['Topic'] == i]
    labels_df = brand.pivot_table(index='Labels1',columns='Sentiment',values='Id',aggfunc='count').reindex(labels_list).reindex(columns = sent_list)
    labels_df_pct_sent = round(labels_df.div(labels_df.sum().sum(),axis=0).fillna(0),3)
    labels_df_pct_sent.index.name = i
    print(labels_df_pct_sent)
    df_list_CORP.append(labels_df_pct_sent)
slide_n_dict['Slide 10'] = df_list_CORP

Sentiment             Positive  Neutral  Negative
Tân Hiệp Phát                                    
HOẠT ĐỘNG FANPAGE        0.000    0.014     0.000
BAN LÃNH ĐẠO             0.000    0.000     0.000
SẢN PHẨM                 0.000    0.000     0.000
THƯƠNG HIỆU/CÔNG TY      0.018    0.528     0.101
CHƯƠNG TRÌNH TÀI TRỢ     0.226    0.058     0.028
QUẢNG CÁO                0.000    0.000     0.000
TUYỂN DỤNG               0.000    0.028     0.000
CHIẾN DỊCH SỰ KIỆN       0.000    0.000     0.000
Sentiment             Positive  Neutral  Negative
Ban Lãnh Đạo Mới                                 
HOẠT ĐỘNG FANPAGE          0.0      0.0       0.0
BAN LÃNH ĐẠO               0.0      0.0       0.0
SẢN PHẨM                   0.0      0.0       0.0
THƯƠNG HIỆU/CÔNG TY        0.0      0.0       0.0
CHƯƠNG TRÌNH TÀI TRỢ       0.0      0.0       0.0
QUẢNG CÁO                  0.0      0.0       0.0
TUYỂN DỤNG                 0.0      0.0       0.0
CHIẾN DỊCH SỰ KIỆN         0.0      0.0       0.0


In [9]:
#Slide 3
slide_3 = []

#===SOV===
for i in topic_SP_list:
    CORP['Topic'] = CORP['Topic'].replace(i,'7 Sản phẩm')
SOV = CORP.groupby('Topic').agg({'Id':'count'})
slide_3.append(SOV)
print(SOV)

#===Sentiment===
sent = CORP.groupby('Sentiment').agg({'Id':'count'}).sort_values(by='Sentiment',ascending = False)
slide_3.append(sent)
print(sent)

#===Sentiment break by Topic===
topic_sent = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id',aggfunc = 'count').reindex(columns= sent_list)
topic_sent = round(topic_sent.div(topic_sent.sum(axis=1),axis=0).fillna(0),3)
slide_3.append(topic_sent)
print(topic_sent)

slide_n_dict['Slide 3'] = slide_3

                 Id
Topic              
7 Sản phẩm     2821
Tân Hiệp Phát  2210
             Id
Sentiment      
Positive   1299
Neutral    3667
Negative     65
Sentiment      Positive  Neutral  Negative
Topic                                     
7 Sản phẩm        0.114    0.886     0.000
Tân Hiệp Phát     0.442    0.529     0.029


In [10]:
# Define the groups with custom names
grouped_topic_dict = {
    'THP&BDL': ['Tân Hiệp Phát', 'Ban Lãnh Đạo Mới'],
    'Trà Xanh Không Độ': ['THP Trà Xanh Không Độ'],
    'Nước tăng Lực Number 1': ['THP Nước tăng Lực Number 1'],
    'Sữa Đậu Nành Soya': ['THP Sữa Đậu Nành Soya'],
    'Trà Thanh Nhiệt Dr Thanh': ['THP Trà Thanh Nhiệt Dr. Thanh'],
    'Trà Sữa Macchiato Không Độ': ['THP Trà Sữa Macchiato Không Độ'],
    'Chanh Muối & Chanh Dâu': ['THP Number 1 Active Chanh Muối', 'THP Number 1 Chanh & Dâu']
}

# Iterate through the dictionary
for group_name, topics in grouped_topic_dict.items():
    df_list_SP = []
    SP_df_working = SP[SP['Topic'].isin(topics)]  # Filter for the topics in the current group
    if SP_df_working.empty:
        print(f"No data available for topics: {group_name}")
        slide_n_dict[group_name] = None  # Or handle it as you prefer (e.g., append an empty list)
        continue  # Skip to the next group if no data
    
    #=======Channel========
    SP_channel = SP_df_working.pivot_table(index='Channel', values='Id', aggfunc='count').reindex(channel_list_SP)
    df_list_SP.append(SP_channel)
    
    #=======Sentiment========
    SP_sentiment = SP_df_working.pivot_table(columns='Sentiment', values='Id', aggfunc='count').reindex(columns=sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_sentiment = round(SP_sentiment.div(SP_sentiment.sum(axis=1), axis=0), 3)
    
    df_list_SP.append(SP_sentiment)
    
    #=======Labels========
    SP_labels = SP_df_working.pivot_table(index='Labels1', columns='Sentiment', values='Id', aggfunc='count').reindex(columns=sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_labels = round(SP_labels.div(SP_labels.sum().sum(), axis=0).fillna(0), 3)
        SP_labels['Total'] = SP_labels.sum(axis=1)
        SP_labels = SP_labels.sort_values(by='Total', ascending=True)
      
    df_list_SP.append(SP_labels)
    
    # Save results for the group using the group name
    slide_n_dict[group_name] = df_list_SP

# Print the keys to check
print(slide_n_dict.keys())


dict_keys(['Slide 6', 'Slide 7', 'Slide 8', 'Slide 9', 'Slide 10', 'Slide 3', 'THP&BDL', 'Trà Xanh Không Độ', 'Nước tăng Lực Number 1', 'Sữa Đậu Nành Soya', 'Trà Thanh Nhiệt Dr Thanh', 'Trà Sữa Macchiato Không Độ', 'Chanh Muối & Chanh Dâu'])


In [11]:
to_excel_file('THPnovOutPut', slide_n_dict)